In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dask
import xarray
import matplotlib.pyplot as plt

import doralite
import gfdl_utils.core as gu
from CM4Xutils import *

### Monthly-mean `zstr` budget diagnostics

In [3]:
sigma2_coords = xr.open_dataset("data/sigma2_coords.nc")
for c in sigma2_coords.dims:
    sigma2_coords.coords[c].attrs = sigma2_coords.coords[c.replace("sigma2", "rho2")].attrs
    sigma2_coords.coords[c].attrs["long_name"] = sigma2_coords.coords[c].attrs["long_name"].replace(
        "Potential Density", "Potential Density minus 1000 kg/m3"
    )
sigma2_coords.coords["sigma2_l"].attrs["edges"] = "sigma2_i"

In [4]:
grid = load_wmt_grid("CM4Xp25", test=True, dmget=True)

Loading CM4Xp25-piControl for interval `2010`.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Loading CM4Xp25-historical for interval `2010`.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Assigning CM4Xp25 grid coordinates.


In [5]:
drop_vars = [
    "obvfsq", "rsdo", "rsdoabsorb", "volcello", "volcello_bounds",
    "uo", "vo", "uhml", "vhml"
]
ds = xr.merge([
    grid._ds.drop_vars(drop_vars),
    sigma2_coords
])

ds.sigma2.attrs = {
    "long_name": "Potential Density referenced to 2000 dbar (minus 1000 kg/m3)",
    "units": "kg m-3",
    "cell_methods": "area:mean z_l:mean yh:mean xh:mean time:mean",
    "volume": "volcello",
    "area": "areacello",
    "time_avg_info": "average_T1,average_T2,average_DT",
    "description": "Computed offline using the gsw python package implementation of TEOS10.",
}
ds.sigma2_bounds.attrs = {
    "long_name": "Potential Density referenced to 2000 dbar (minus 1000 kg/m3)",
    "units": "kg m-3",
    "cell_methods": "area:mean z_l:mean yh:mean xh:mean time:point",
    "volume": "volcello",
    "area": "areacello",
    "description": "Computed offline using the gsw python package implementation of TEOS10."
}

### Transform all variables to density coordinates

In [7]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=UserWarning)
    
    ds_sigma2 = xr.merge([remap_to_sigma2(ds, grid), ds[["tos", "sos"]]])
    grid_sigma2 = ds_to_grid(ds_sigma2)

In [8]:
ds_sigma2

<xarray.Dataset> Size: 314GB
Dimensions:                         (exp: 2, nv: 2, xh: 1440, yh: 1080,
                                     xq: 1441, yq: 1081, time: 12,
                                     time_bounds: 13, sigma2_l: 74, sigma2_i: 75)
Coordinates: (12/31)
  * exp                             (exp) object 16B 'forced' 'control'
  * nv                              (nv) float64 16B 1.0 2.0
  * xh                              (xh) float64 12kB -299.7 -299.5 ... 60.03
  * yh                              (yh) float64 9kB -80.39 -80.31 ... 89.95
  * xq                              (xq) float64 12kB -299.8 -299.6 ... 60.16
  * yq                              (yq) float64 9kB -80.43 -80.35 ... 90.0
    ...                              ...
    wet_u                           (yh, xq) float32 6MB 0.0 0.0 0.0 ... 0.0 0.0
    wet                             (yh, xh) float32 6MB 0.0 0.0 0.0 ... 0.0 0.0
  * sigma2_l                        (sigma2_l) float64 592B 4.246 ... 38.49
  * sigma2_i                        (sigma2_i) float64 600B -3.0 11.49 ... 39.0
    rho2_i                          (sigma2_i) float64 600B ...
    rho2_l                          (sigma2_l) float64 592B ...
Data variables: (12/29)
    thkcello                        (exp, time, yh, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1440, 74), meta=np.ndarray>
    thkcello_bounds                 (exp, yh, xh, time_bounds, sigma2_l) float32 12GB dask.array<chunksize=(1, 1080, 1440, 1, 74), meta=np.ndarray>
    S_advection_xy                  (exp, time, yh, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1440, 74), meta=np.ndarray>
    Sh_tendency_vert_remap          (exp, time, yh, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1440, 74), meta=np.ndarray>
    T_advection_xy                  (exp, time, yh, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1440, 74), meta=np.ndarray>
    Th_tendency_vert_remap          (exp, time, yh, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1440, 74), meta=np.ndarray>
    ...                              ...
    thetao_bounds                   (exp, yh, xh, time_bounds, sigma2_l) float32 12GB dask.array<chunksize=(1, 1080, 1440, 1, 74), meta=np.ndarray>
    sigma2_bounds                   (exp, yh, xh, time_bounds, sigma2_l) float64 24GB dask.array<chunksize=(1, 1080, 1440, 1, 74), meta=np.ndarray>
    umo                             (exp, time, yh, xq, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1080, 1441, 74), meta=np.ndarray>
    vmo                             (exp, time, yq, xh, sigma2_l) float32 11GB dask.array<chunksize=(1, 1, 1081, 1440, 74), meta=np.ndarray>
    tos                             (exp, time, yh, xh) float32 149MB dask.array<chunksize=(1, 1, 1080, 1440), meta=np.ndarray>
    sos                             (exp, time, yh, xh) float32 149MB dask.array<chunksize=(1, 1, 1080, 1440), meta=np.ndarray>
Attributes:
    filename:            ocean_monthly_z.201001-201412.S_advection_xy.nc
    title:               CM4_historical_c192_OM4p25
    associated_files:    areacello: 20100101.ocean_static.nc
    grid_type:           regular
    grid_tile:           N/A
    external_variables:  volcello areacello
    model:               CM4Xp25
    description:         The CM4Xp25 experimental design following Griffies e...
    provenance:          Diagnostics have been conservatively remapped into m...

### Horizontal coarsening

In [9]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_sigma2_coarse = horizontally_coarsen(ds_sigma2, grid_sigma2, dim = {"X":12, "Y":12})

In [10]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_coarse = horizontally_coarsen(ds, grid, dim = {"X":12, "Y":12})

Skipping variable average_DT because `cell_methods` not defined.
Skipping variable average_T1 because `cell_methods` not defined.
Skipping variable average_T2 because `cell_methods` not defined.
Skipping variable time_bnds because `cell_methods` not defined.
Skipping variable sigma2_u because `cell_methods` not defined.
Skipping variable sigma2_v because `cell_methods` not defined.


### Tracers

In [506]:
pp = doralite.dora_metadata(exp_dict["CM4Xp125"]["piControl"])["pathPP"]
gu.get_allvars(pp)

{'ocean_inert_z_1x1deg': ['cfc11', 'cfc12', 'sf6', 'volcello'],
 'ocean_inert_z': ['cfc11', 'cfc12', 'sf6', 'volcello'],
 'ocean_inert_month_1x1deg': ['cfc12_csurf',
  'cfc12',
  'cfc12_sc_no',
  'cfc12_stf_gas',
  'fgcfc11',
  'fgcfc12',
  'fgsf6',
  'sf6_sc_no',
  'volcello',
  'sf6_stf_gas',
  'sf6_alpha',
  'sf6_csurf',
  'sf6',
  'cfc11_alpha',
  'cfc11_csurf',
  'cfc11',
  'cfc11_sc_no',
  'cfc11_stf_gas',
  'cfc12_alpha'],
 'ocean_inert_month': ['fgsf6',
  'cfc12_alpha',
  'cfc12_csurf',
  'cfc12',
  'cfc12_sc_no',
  'cfc12_stf_gas',
  'fgcfc11',
  'fgcfc12',
  'sf6_alpha',
  'sf6_csurf',
  'sf6',
  'sf6_sc_no',
  'sf6_stf_gas',
  'volcello',
  'cfc11_alpha',
  'cfc11_csurf',
  'cfc11',
  'cfc11_sc_no',
  'cfc11_stf_gas'],
 'atmos_cmip': ['ci',
  'clivi',
  'clt',
  'clwvi',
  'evspsbl',
  'hfls',
  'hfss',
  'hurs',
  'hur_unmsk',
  'huss',
  'hus_unmsk',
  'o3_unmsk',
  'prc',
  'pr',
  'prsn',
  'prw',
  'psl',
  'ps',
  'rldscs',
  'rlds',
  'rlus',
  'rlutcs',
  'rlut',
  '

In [507]:
v = "pbo"
ppname = gu.find_variable(pp, v)[0]
out = "ts"
local = gu.get_local(pp, ppname, out)

In [508]:
ds = gu.open_frompp(pp, ppname, out, local, "0101*", v, dmget=True)

Issuing dmget command to migrate data to disk. Migration complete.


In [509]:
ds

<xarray.Dataset> Size: 258MB
Dimensions:     (time: 5, nv: 2, yh: 2240, xh: 2880)
Coordinates:
  * nv          (nv) float64 16B 1.0 2.0
  * time        (time) object 40B 0101-07-02 12:00:00 ... 0105-07-02 12:00:00
  * xh          (xh) float64 23kB -298.6 -298.5 -298.4 ... 61.05 61.16 61.28
  * yh          (yh) float64 18kB -83.62 -83.58 -83.54 ... 89.86 89.92 89.97
Data variables:
    average_DT  (time) timedelta64[ns] 40B dask.array<chunksize=(5,), meta=np.ndarray>
    average_T1  (time) object 40B dask.array<chunksize=(5,), meta=np.ndarray>
    average_T2  (time) object 40B dask.array<chunksize=(5,), meta=np.ndarray>
    pbo         (time, yh, xh) float64 258MB dask.array<chunksize=(1, 560, 720), meta=np.ndarray>
    time_bnds   (time, nv) object 80B dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    filename:            ocean_annual.0101-0105.pbo.nc
    title:               CM4_piControl_c192_OM4p125_v8
    associated_files:    areacello: 01010101.ocean_static.nc
    grid_type:           regular
    grid_tile:           N/A
    external_variables:  areacello

In [444]:
gu.find_variable(pp, v)

['ocean_annual', 'ocean_monthly_1x1deg', 'ocean_monthly']

In [445]:
gu.find_variable(doralite.dora_metadata(exp_dict["CM4Xp25"]["piControl-spinup"])["pathPP"], "agessc")

['ocean_annual_z_1x1deg', 'ocean_annual_z']

In [426]:
gu.find_variable(doralite.dora_metadata(exp_dict["CM4Xp25"]["piControl"])["pathPP"], "agessc")

['ocean_annual_z_1x1deg', 'ocean_annual_z']

In [429]:
gu.find_variable(doralite.dora_metadata(exp_dict["CM4Xp125"]["piControl"])["pathPP"], "agessc")

['ocean_annual_z_d2_1x1deg', 'ocean_annual_z_d2']

In [430]:
gu.find_variable(doralite.dora_metadata(exp_dict["CM4Xp125"]["piControl"])["pathPP"], "cfc11")

['ocean_inert_z_1x1deg',
 'ocean_inert_z',
 'ocean_inert_month_1x1deg',
 'ocean_inert_month']